In [2]:
import os
import numpy as np
import pandas as pd

In [41]:
datasets = ["MoonsDataset", "LawDataset", "WineDataset", "BlobsDataset"]
disc_models = ["LogisticRegression", "MultilayerPerceptron"]
methods = ["ARES", "GCE", "GLOBAL_CE", "PPCEF_2"]
df = pd.DataFrame()
for dataset in datasets:
    metric_files = [
        file
        for file in os.listdir(f"../models/{dataset}/GCE_CV/")
        if file.startswith("metrics") and not file.startswith("metrics_no")
    ]
    for metric_file in metric_files:
        df_metrics = pd.read_csv(f"../models/{dataset}/GCE_CV/{metric_file}")
        metric_file = metric_file.replace("GLOBAL_CE", "GLOBAL-CE")
        # print(metric_file.split("_"))
        method = metric_file.split("_")[2]
        disc_model = metric_file.split("_")[1]
        origin_class = metric_file.split("_")[-2]

        df_metrics["dataset"] = dataset
        df_metrics["disc_model"] = disc_model
        df_metrics["method"] = method
        df_metrics["origin_class"] = int(origin_class)

        df = pd.concat([df, df_metrics])


# f"models/{dataset}/GCE_CV/metrics_{disc_model}_{method}_{origin_class}_cv.csv"

In [42]:
df

,model_returned_smth,valid_cf_disc,dissimilarity_proximity_categorical_hamming,dissimilarity_proximity_categorical_jaccard,dissimilarity_proximity_continuous_manhatan,dissimilarity_proximity_continuous_euclidean,dissimilarity_proximity_continuous_mad,distance_l2_jaccard,distance_mad_hamming,sparsity,...,isolation_forest_scores_xs,isolation_forest_scores_cfs,plausibility,cf_belongs_to_group,K_vectors,time,dataset,disc_model,method,origin_class
0,1.0,0.411765,NaN,0.0,0.325699,0.299830,4.755227,0.299830,NaN,1.0,...,0.000427,-0.012588,0.047941,1.0,1,6.601301,MoonsDataset,MultilayerPerceptron,ARES,0
1,1.0,0.576923,NaN,0.0,0.365293,0.327727,4.483185,0.327727,NaN,1.0,...,0.003519,-0.017831,0.039109,1.0,1,6.571543,MoonsDataset,MultilayerPerceptron,ARES,0
2,1.0,0.728155,NaN,0.0,0.592190,0.419311,4.217731,0.419311,NaN,1.0,...,0.001189,-0.016877,0.060560,1.0,1,14.356016,MoonsDataset,MultilayerPerceptron,ARES,0
3,1.0,0.607843,NaN,0.0,0.387485,0.286907,3.259045,0.286907,NaN,1.0,...,-0.001777,-0.011648,0.082561,1.0,1,7.869122,MoonsDataset,MultilayerPerceptron,ARES,0
4,1.0,0.805825,NaN,0.0,0.714046,0.524478,5.601976,0.524478,NaN,1.0,...,0.000197,-0.033562,0.082923,1.0,1,15.373326,MoonsDataset,MultilayerPerceptron,ARES,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1.0,1.000000,NaN,0.0,0.612890,0.434905,14.615149,0.434905,NaN,1.0,...,0.025462,0.031619,0.014159,1.0,1,8.291227,BlobsDataset,MultilayerPerceptron,GLOBAL-CE,0
1,1.0,1.000000,NaN,0.0,0.579851,0.413726,12.969225,0.413726,NaN,1.0,...,0.018060,0.027858,0.010602,1.0,1,7.345126,BlobsDataset,MultilayerPerceptron,GLOBAL-CE,0
2,1.0,0.990000,NaN,0.0,0.576603,0.412547,14.522619,0.412547,NaN,1.0,...,0.027420,0.023327,0.016720,1.0,1,14.779735,BlobsDataset,MultilayerPerceptron,GLOBAL-CE,0
3,1.0,1.000000,NaN,0.0,0.596985,0.427569,13.753640,0.427569,NaN,1.0,...,0.022112,0.030900,0.011097,1.0,1,19.316764,BlobsDataset,MultilayerPerceptron,GLOBAL-CE,0


In [43]:
column_mapping = {
    "dataset": "Dataset",
    "disc_model": "Model",
    "method": "Method",
    "origin_class": "Origin Class",
    "K_vectors": "K",
    "valid_cf_disc": "Validity",
    "flow_prob_condition_acc": "Prob. Plaus.",
    "cf_belongs_to_group": "CFs assigned to group",
    "flow_log_density_cfs": "Log Dens.",
    "dissimilarity_proximity_continuous_manhatan": "L1",
    "dissimilarity_proximity_continuous_euclidean": "L2",
    "isolation_forest_scores_cfs": "IsoForest",
    "lof_scores_cfs": "LOF",
    # 'plausibility': 'Plausibility',
    "time": "Time",
}

df = df.rename(columns=column_mapping)[column_mapping.values()]
df["Model"] = df["Model"].replace(
    {"MultinomialLogisticRegression": "LogisticRegression"}
)

df["Method"] = df["Method"].replace(
    {
        "ARES": "GCE",
        "GCE": "OUR",
        "PPCEF": "LCF",
        "GLOBAL-CE": "GCE w/ magn.",
    }
)
df["Model"] = df["Model"].replace(
    {
        "LogisticRegression": "LR",
        "MultilayerPerceptron": "MLP",
    }
)
df["Dataset"] = df["Dataset"].apply(lambda x: x[:-7])
df = df[df["Origin Class"] == 0]
df_mean = (
    df.groupby(["Model", "Dataset", "Method"], as_index=True)
    .mean()
    .round(2)
    .drop(columns=["Origin Class"])
    .sort_index(level=["Model", "Dataset", "Method"])
    # .drop(columns=["Method"])
    # .drop(columns=["K"])
)
df_std = (
    df.groupby(["Model", "Dataset", "Method"], as_index=True)
    .std()
    .round(2)
    .drop(columns=["Origin Class"])
    .sort_index(level=["Model", "Dataset", "Method"])
    # .drop(columns=["Method"])
    # .drop(columns=["K"])
)

In [44]:
df_mean[
    [
        "K",
        "Validity",
        "Prob. Plaus.",
        "CFs assigned to group",
        "Log Dens.",
        "L1",
        "L2",
        "Time",
    ]
]

K  Validity  Prob. Plaus.  \
Model Dataset Method                                        
LR    Blobs   GCE             1.0      1.00          0.75   
              GCE w/ magn.    1.0      1.00          0.92   
              LCF           100.4      1.00          1.00   
              OUR             5.8      1.00          0.98   
      Law     GCE             1.0      0.97          0.58   
              GCE w/ magn.    1.0      1.00          0.83   
              LCF           223.0      1.00          1.00   
              OUR            34.4      1.00          0.98   
      Moons   GCE             1.0      0.89          0.36   
              GCE w/ magn.    1.0      0.89          0.50   
              LCF           102.8      1.00          1.00   
              OUR            22.2      0.98          0.87   
      Wine    GCE             1.0      0.97          0.79   
              GCE w/ magn.    1.0      1.00          0.88   
              LCF            11.8      1.00          1.00   
              OUR             4.4      1.00          1.00   
MLP   Blobs   GCE             1.0      1.00          0.75   
              GCE w/ magn.    1.0      1.00          0.93   
              LCF           100.0      1.00          1.00   
              OUR             9.2      1.00          0.99   
      Law     GCE             1.0      0.90          0.62   
              GCE w/ magn.    1.0      0.93          0.81   
              LCF           236.4      0.96          1.00   
              OUR            37.4      0.95          0.98   
      Moons   GCE             1.0      0.63          0.23   
              GCE w/ magn.    1.0      0.79          0.74   
              LCF           102.8      0.99          1.00   
              OUR            19.6      0.99          0.98   
      Wine    GCE             1.0      0.95          0.78   
              GCE w/ magn.    1.0      0.98          0.90   
              LCF            11.8      0.98          1.00   
              OUR             4.0      0.98          1.00   

                            CFs assigned to group  Log Dens.    L1    L2  \
Model Dataset Method                                                       
LR    Blobs   GCE                            1.00       2.60  0.64  0.46   
              GCE w/ magn.                   1.00       2.84  0.60  0.43   
              LCF                            1.00       2.25  0.49  0.35   
              OUR                            0.99       2.79  0.60  0.43   
      Law     GCE                            1.00       0.93  0.72  0.43   
              GCE w/ magn.                   1.00       1.73  0.42  0.25   
              LCF                            1.00       1.84  0.35  0.21   
              OUR                            1.00       2.01  0.48  0.29   
      Moons   GCE                            1.00      -1.24  0.66  0.47   
              GCE w/ magn.                   1.00       0.80  0.50  0.37   
              LCF                            1.00       1.26  0.45  0.35   
              OUR                            0.99       1.32  0.51  0.39   
      Wine    GCE                            1.00       8.54  2.16  0.72   
              GCE w/ magn.                   1.00       8.76  1.93  0.65   
              LCF                            1.00       8.16  1.46  0.46   
              OUR                            1.00       8.80  1.97  0.63   
MLP   Blobs   GCE                            1.00       2.62  0.64  0.46   
              GCE w/ magn.                   1.00       2.83  0.59  0.42   
              LCF                            1.00       2.31  0.50  0.35   
              OUR                            1.00       2.83  0.60  0.43   
      Law     GCE                            1.00       0.95  0.67  0.41   
              GCE w/ magn.                   1.00       1.78  0.50  0.30   
              LCF                            1.00       2.19  0.45  0.27   
              OUR                            0.99       2

In [45]:
print(
    df_mean[
        [
            "K",
            "Validity",
            "Prob. Plaus.",
            "CFs assigned to group",
            "Log Dens.",
            "L1",
            "L2",
            "Time",
        ]
    ].to_latex(multirow=True, escape=False, float_format="%.2f")
)

\begin{tabular}{lllrrrrrrrr}
\toprule
 &  &  & K & Validity & Prob. Plaus. & CFs assigned to group & Log Dens. & L1 & L2 & Time \\
Model & Dataset & Method &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{16}{*}{LR} & \multirow[t]{4}{*}{Blobs} & GCE & 1.00 & 1.00 & 0.75 & 1.00 & 2.60 & 0.64 & 0.46 & 21.95 \\
 &  & GCE w/ magn. & 1.00 & 1.00 & 0.92 & 1.00 & 2.84 & 0.60 & 0.43 & 21.63 \\
 &  & LCF & 100.40 & 1.00 & 1.00 & 1.00 & 2.25 & 0.49 & 0.35 & 12.11 \\
 &  & OUR & 5.80 & 1.00 & 0.98 & 0.99 & 2.79 & 0.60 & 0.43 & 102.75 \\
\cline{2-11}
 & \multirow[t]{4}{*}{Law} & GCE & 1.00 & 0.97 & 0.58 & 1.00 & 0.93 & 0.72 & 0.43 & 17.53 \\
 &  & GCE w/ magn. & 1.00 & 1.00 & 0.83 & 1.00 & 1.73 & 0.42 & 0.25 & 61.19 \\
 &  & LCF & 223.00 & 1.00 & 1.00 & 1.00 & 1.84 & 0.35 & 0.21 & 20.00 \\
 &  & OUR & 34.40 & 1.00 & 0.98 & 1.00 & 2.01 & 0.48 & 0.29 & 225.35 \\
\cline{2-11}
 & \multirow[t]{4}{*}{Moons} & GCE & 1.00 & 0.89 & 0.36 & 1.00 & -1.24 & 0.66 & 0.47 & 45.23 \\
 &  & GCE w/ magn. & 1.00 & 0.

In [46]:
# group_cols = ["Model", "Dataset", "Method"]
# df_mean_cols = [c+"_mean" for c in df_mean.columns if c not in group_cols]
# df_std_cols = [c+"_std" for c in df_std.columns if c not in group_cols]
# df_mean.columns = df_mean_cols
# df_std.columns = df_std_cols

# sum([list(x) for x in zip(df_mean.columns, df_std.columns)], [])
# df_result = pd.merge(df_mean, df_std, on=["Model", "Dataset", "Method"])[sum([list(x) for x in zip(df_mean.columns, df_std.columns)], [])]


In [47]:
for column in df_mean.columns:
    df_mean[column] = (
        "$ " + df_mean[column].astype(str) + " \pm " + df_std[column].astype(str) + " $"
    )
df_mean.to_csv("plaus.csv")

In [48]:
print(df_mean.to_latex(multirow=True, escape=False, float_format="%.2f"))

\begin{tabular}{lllllllllllll}
\toprule
 &  &  & K & Validity & Prob. Plaus. & CFs assigned to group & Log Dens. & L1 & L2 & IsoForest & LOF & Time \\
Model & Dataset & Method &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{16}{*}{LR} & \multirow[t]{4}{*}{Blobs} & GCE & $ 1.0 \pm 0.0 $ & $ 1.0 \pm 0.0 $ & $ 0.75 \pm 0.04 $ & $ 1.0 \pm 0.0 $ & $ 2.6 \pm 0.07 $ & $ 0.64 \pm 0.02 $ & $ 0.46 \pm 0.01 $ & $ 0.02 \pm 0.0 $ & $ 1.09 \pm 0.01 $ & $ 21.95 \pm 4.29 $ \\
 &  & GCE w/ magn. & $ 1.0 \pm 0.0 $ & $ 1.0 \pm 0.0 $ & $ 0.92 \pm 0.03 $ & $ 1.0 \pm 0.0 $ & $ 2.84 \pm 0.08 $ & $ 0.6 \pm 0.02 $ & $ 0.43 \pm 0.01 $ & $ 0.03 \pm 0.0 $ & $ 1.05 \pm 0.01 $ & $ 21.63 \pm 4.78 $ \\
 &  & LCF & $ 100.4 \pm 0.55 $ & $ 1.0 \pm 0.0 $ & $ 1.0 \pm 0.0 $ & $ 1.0 \pm 0.0 $ & $ 2.25 \pm 0.05 $ & $ 0.49 \pm 0.02 $ & $ 0.35 \pm 0.01 $ & $ 0.02 \pm 0.01 $ & $ 1.11 \pm 0.04 $ & $ 12.11 \pm 2.18 $ \\
 &  & OUR & $ 5.8 \pm 0.45 $ & $ 1.0 \pm 0.0 $ & $ 0.98 \pm 0.01 $ & $ 0.99 \pm 0.01 $ & $ 2.79 \pm 0.27

In [91]:
df = pd.read_csv("../Counterfactuals_lambda_ablation.csv")

column_mapping = {
    "parameters/method": "Method",
    "parameters/disc_model/model_name": "Model",
    "parameters/dataset": "Dataset",
    "parameters/counterfactuals/origin_class": "Origin Class",
    # "parameters/counterfactuals/K": "K",
    "metrics/cf/K_vectors": "K",
    "metrics/cf/valid_cf_disc": "Validity",
    "metrics/cf/flow_prob_condition_acc": "Prob. Plaus.",
    # "metrics/cf/cf_belongs_to_group": "CFs assigned to group",
    # "metrics/cf/flow_log_density_cfs": "Log Dens.",
    # "metrics/cf/dissimilarity_proximity_continuous_manhatan": "L1",
    # "metrics/cf/dissimilarity_proximity_continuous_euclidean": "L2",
    # "metrics/cf/isolation_forest_scores_cfs": "IsoForest",
    # "metrics/cf/lof_scores_cfs": "LOF",
    # "metrics/cf/plausibility": "Plaus.",
    "parameters/counterfactuals/alpha": "Alpha",
    "parameters/counterfactuals/alpha_k": "Alpha K",
    "parameters/counterfactuals/alpha_s": "Alpha S",
}
df = df.rename(columns=column_mapping)[column_mapping.values()]
df["Model"] = df["Model"].replace(
    {"MultinomialLogisticRegression": "LogisticRegression"}
)
df["Model"] = df["Model"].replace(
    {
        "LogisticRegression": "LR",
        "MultilayerPerceptron": "MLP",
    }
)
df["Dataset"] = df["Dataset"].apply(lambda x: x[:-7])
df = (
    df.groupby(
        ["Model", "Dataset", "Method", "Alpha", "Alpha K", "Alpha S"], as_index=False
    )
    .mean()
    .round(2)
    .drop(columns=["Origin Class"])
    .drop(columns=["Method"])
    .drop(columns=["K"])
)

In [92]:
df

,Model,Dataset,Alpha,Alpha K,Alpha S,Validity,Prob. Plaus.
0,LR,Blobs,1,1,1,1.0,0.91
1,LR,Blobs,1,1,10,1.0,0.88
2,LR,Blobs,1,1,100,1.0,0.91
3,LR,Blobs,1,1,1000,1.0,0.93
4,LR,Blobs,1,10,1,1.0,0.88
...,...,...,...,...,...,...,...
507,MLP,Wine,1000,100,1000,1.0,1.00
508,MLP,Wine,1000,1000,1,1.0,0.98
509,MLP,Wine,1000,1000,10,1.0,1.00
510,MLP,Wine,1000,1000,100,1.0,1.00


In [90]:
print(
    df[(df["Validity"] >= 0.95) & (df["Prob. Plaus."] >= 0.95)]
    .groupby(["Model", "Dataset"])
    .mean()
    .round(2)
    .to_latex(multirow=True, escape=False, float_format="%.2f")
)

\begin{tabular}{llrrrrr}
\toprule
 &  & Alpha & Alpha K & Alpha S & Validity & Prob. Plaus. \\
Model & Dataset &  &  &  &  &  \\
\midrule
\multirow[t]{4}{*}{LR} & Blobs & 489.03 & 90.71 & 286.39 & 1.00 & 0.98 \\
 & Law & 459.36 & 18.36 & 316.64 & 1.00 & 0.99 \\
 & Moons & 635.50 & 28.90 & 178.30 & 1.00 & 0.98 \\
 & Wine & 464.14 & 130.65 & 297.51 & 1.00 & 0.99 \\
\cline{1-7}
\multirow[t]{4}{*}{MLP} & Blobs & 465.14 & 112.34 & 257.11 & 1.00 & 0.98 \\
 & Law & 498.14 & 70.00 & 316.00 & 0.95 & 0.99 \\
 & Moons & 480.00 & 137.00 & 277.75 & 0.99 & 0.99 \\
 & Wine & 443.15 & 149.85 & 310.00 & 1.00 & 0.99 \\
\cline{1-7}
\bottomrule
\end{tabular}



In [ ]:
print(df[df["Model"] == "MLP"].to_latex(index=False, float_format="%.2f"))

\begin{tabular}{lllrrrrrrrrr}
\toprule
Model & Dataset & Method & K & Validity & Prob. Plaus. & CFs assigned to group & Log Dens. & L1 & L2 & IsoForest & LOF \\
\midrule
MLP & Blobs & GCE & 1 & 1.00 & 0.93 & 1.00 & 2.84 & 0.70 & 0.50 & 0.03 & 1.05 \\
MLP & Blobs & GCE & 2 & 1.00 & 0.94 & 1.00 & 2.84 & 0.70 & 0.50 & 0.03 & 1.05 \\
MLP & Blobs & GCE & 4 & 1.00 & 0.94 & 1.00 & 2.89 & 0.70 & 0.51 & 0.03 & 1.04 \\
MLP & Blobs & GCE & 6 & 1.00 & 0.96 & 1.00 & 2.87 & 0.71 & 0.51 & 0.03 & 1.04 \\
MLP & Blobs & GCE & 8 & 1.00 & 0.93 & 0.98 & 2.66 & 0.69 & 0.50 & 0.01 & 1.06 \\
MLP & Blobs & GCE & 10 & 1.00 & 0.99 & 0.99 & 2.83 & 0.71 & 0.51 & 0.02 & 1.05 \\
MLP & Blobs & GCE & 20 & 1.00 & 0.99 & 1.00 & 2.97 & 0.73 & 0.53 & 0.03 & 1.03 \\
MLP & Blobs & GCE & 30 & 1.00 & 0.98 & 1.00 & 2.96 & 0.73 & 0.52 & 0.03 & 1.04 \\
MLP & Blobs & GCE & 40 & 1.00 & 0.97 & 0.99 & 2.83 & 0.69 & 0.50 & 0.02 & 1.05 \\
MLP & Blobs & GCE & 50 & 1.00 & 0.98 & 1.00 & 2.94 & 0.71 & 0.51 & 0.03 & 1.04 \\
MLP & Blobs & G

In [ ]:
dataset_name = "MoonsDataset"
experiment_output_folder = "../models"
METHOD = "ppcef"
disc_model = "MultinomialLogisticRegression"  # MultilayerPerceptron, MultinomialLogisticRegression, LogisticRegression, NODE
columns = {
    "dataset": "dataset",
    "method": "method",
    "model_returned_smth": "Coverage",
    "valid_cf_disc": "validity",
    "flow_prob_condition_acc": "Prob. Plaus.",
    "lof_scores_cfs": "LOF",
    # "lof_scores_xs": "LOF_x",
    "isolation_forest_scores_cfs": "IsoForest",
    # "isolation_forest_scores_xs": "IsoForest_x",
    "flow_log_density_cfs": "Log Dens.",
    "dissimilarity_proximity_continuous_manhatan": "L1",
    "dissimilarity_proximity_continuous_euclidean": "L2",
    "time": "Time",
}

In [ ]:
# First batch
datasets = [
    "MoonsDataset",
    "LawDataset",
    "AuditDataset",
    "HelocDataset",
    # "BlobsDataset",
    # "DigitsDataset",
    # "WineDataset",
]
methods = [
    # "cbce",
    # "CEGP",
    # "CEM",
    # "wach",
    "ppcef",
    # "artelth20"
]  #  ,
df_results = pd.DataFrame()
for dataset_name in datasets:
    for method in methods:
        output_folder = os.path.join(experiment_output_folder, dataset_name)
        os.makedirs(output_folder, exist_ok=True)
        save_folder = os.path.join(output_folder, method)
        os.makedirs(save_folder, exist_ok=True)

        df_part_results = pd.read_csv(
            os.path.join(save_folder, f"metrics_{disc_model}_cv.csv")
        )
        means = df_part_results.iloc[0]
        output = {k: None for k in columns.values()}
        # output = dict()
        for key, value in columns.items():
            if value == "IsoForest":
                output[value] = f"{means.get(key, np.nan):.3f}"
            elif value == "LOF":
                if means.get(key, float("inf")) < 10:
                    output[value] = f"{means.get(key, np.nan):.2f}"
                else:
                    output[value] = f"{means.get(key, np.nan):.2e}"
            else:
                output[value] = f"{means.get(key, np.nan):.2f}"
        output["dataset"] = dataset_name.removesuffix("Dataset")
        output["method"] = method
        df_part_results = (
            pd.Series(output).to_frame().T.rename(columns=columns)[columns.values()]
        )

        df_results = pd.concat([df_results, df_part_results], axis=0, ignore_index=True)

In [ ]:
df_results

In [ ]:
print(df_results.to_latex(index=False))

In [ ]:
# All batches
datasets = [
    # "MoonsDataset",
    # "LawDataset",
    # "AuditDataset",
    # "HelocDataset",
    "BlobsDataset",
    "DigitsDataset",
    "WineDataset",
]
methods = ["ppcef"]  # "artelth20","cbce", "CEM","CEGP","wach",
df_results = pd.DataFrame()
for dataset_name in datasets:
    for method in methods:
        output_folder = os.path.join(experiment_output_folder, dataset_name)
        os.makedirs(output_folder, exist_ok=True)
        save_folder = os.path.join(output_folder, method)
        os.makedirs(save_folder, exist_ok=True)

        df_part_results = pd.read_csv(
            os.path.join(save_folder, f"metrics_{disc_model}_CE_cv.csv")
        )
        means = df_part_results.mean().round(3).to_dict()
        stds = df_part_results.std().round(3).to_dict()
        output = {k: None for k in columns.values()}
        # output = dict()
        for key, value in columns.items():
            if value == "IsoForest":
                output[value] = (
                    f"{means.get(key, np.nan):.3f}$\pm${stds.get(key, np.nan):.3f}"
                )
            elif value == "LOF":
                if means.get(key, float("inf")) < 10:
                    output[value] = (
                        f"{means.get(key, np.nan):.2f}$\pm${stds.get(key, np.nan):.2f}"
                    )
                else:
                    output[value] = (
                        f"{means.get(key, np.nan):.2e}$\pm${stds.get(key, np.nan):.2e}"
                    )
            else:
                output[value] = (
                    f"{means.get(key, np.nan):.2f}$\pm${stds.get(key, np.nan):.2f}"
                )
        output["dataset"] = dataset_name.removesuffix("Dataset")
        output["method"] = method
        df_part_results = (
            pd.Series(output).to_frame().T.rename(columns=columns)[columns.values()]
        )

        df_results = pd.concat([df_results, df_part_results], axis=0, ignore_index=True)

In [ ]:
df_results

In [ ]:
df_results

In [ ]:
print(df_results.to_latex(index=False))

In [ ]:
df_res = pd.DataFrame()
for i in [1, 2, 5, 10, 100, 1000]:
    df = pd.read_csv(
        f"../models/LawDataset/ppcef/metrics_LogisticRegression_lambda_{i}_cv.csv"
    ).mean()
    df["lambda"] = i
    df_res = pd.concat([df_res, df.to_frame()], axis=1)

In [ ]:
columns = {
    "lambda": "lambda",
    # "dataset": "dataset",
    # "method": "method",
    "model_returned_smth": "Coverage",
    "valid_cf_disc": "validity",
    "flow_prob_condition_acc": "Prob. Plaus.",
    "lof_scores_cfs": "LOF",
    # "lof_scores_xs": "LOF_x",
    "isolation_forest_scores_cfs": "IsoForest",
    # "isolation_forest_scores_xs": "IsoForest_x",
    "flow_log_density_cfs": "Log Dens.",
    "dissimilarity_proximity_continuous_manhatan": "L1",
    "dissimilarity_proximity_continuous_euclidean": "L2",
    "time": "Time",
}

df_res.T.rename(columns=columns)[columns.values()].round(2)

In [8]:
column_mapping = {
    # "dataset": "Dataset",
    # "disc_model": "Model",
    # "method": "Method",
    # "origin_class": "Origin Class",
    "K_vectors": "K",
    # "valid_cf_disc": "Validity",
    "flow_prob_condition_acc": "Prob. Plaus.",
    # "cf_belongs_to_group": "CFs assigned to group",
    "dissimilarity_proximity_continuous_manhatan": "L1",
    "dissimilarity_proximity_continuous_euclidean": "L2",
    "isolation_forest_scores_cfs": "IsoForest",
    "lof_scores_cfs": "LOF",
    "flow_log_density_cfs": "Log Dens.",
    # 'plausibility': 'Plausibility',
    # "time": "Time",
}

df = pd.read_csv("group_results/metrics_GCE_groups.csv", index_col=0)
print(
    df.rename(columns=column_mapping)[column_mapping.values()].to_markdown(
        index=False, floatfmt=".2f"
    )
)

|    K |   Prob. Plaus. |   L1 |   L2 |   IsoForest |   LOF |   Log Dens. |
|-----:|---------------:|-----:|-----:|------------:|------:|------------:|
| 2.00 |           0.91 | 0.60 | 0.46 |        0.02 |  1.04 |        1.66 |
| 3.00 |           0.96 | 0.58 | 0.45 |        0.02 |  1.04 |        1.63 |
| 4.00 |           0.96 | 0.50 | 0.41 |        0.02 |  1.02 |        1.75 |
| 5.00 |           1.00 | 0.58 | 0.45 |        0.02 |  1.01 |        1.83 |
| 6.00 |           1.00 | 0.52 | 0.42 |        0.02 |  1.02 |        1.81 |
| 7.00 |           0.96 | 0.49 | 0.40 |        0.02 |  1.02 |        1.72 |
| 8.00 |           0.98 | 0.51 | 0.43 |        0.02 |  1.01 |        1.81 |
| 7.00 |           1.00 | 0.51 | 0.42 |        0.02 |  1.02 |        1.81 |
| 9.00 |           1.00 | 0.48 | 0.40 |        0.02 |  1.01 |        1.77 |


In [9]:
df = pd.read_csv("group_results/metrics_GCE_k_means_groups.csv", index_col=0)
print(
    df.rename(columns=column_mapping)[column_mapping.values()].to_markdown(
        index=False, floatfmt=".2f"
    )
)

|     K |   Prob. Plaus. |   L1 |   L2 |   IsoForest |   LOF |   Log Dens. |
|------:|---------------:|-----:|-----:|------------:|------:|------------:|
|  2.00 |           0.76 | 0.46 | 0.37 |        0.01 |  1.20 |        0.87 |
|  3.00 |           0.96 | 0.58 | 0.43 |        0.02 |  1.03 |        1.75 |
|  4.00 |           0.93 | 0.43 | 0.34 |        0.03 |  1.05 |        1.61 |
|  5.00 |           0.93 | 0.42 | 0.34 |        0.03 |  1.05 |        1.62 |
|  6.00 |           0.93 | 0.42 | 0.34 |        0.04 |  1.04 |        1.73 |
|  7.00 |           1.00 | 0.40 | 0.31 |        0.04 |  1.02 |        1.78 |
|  8.00 |           1.00 | 0.41 | 0.32 |        0.04 |  1.01 |        1.79 |
|  9.00 |           1.00 | 0.39 | 0.30 |        0.04 |  1.01 |        1.77 |
| 10.00 |           1.00 | 0.40 | 0.30 |        0.04 |  1.01 |        1.75 |


In [10]:
df = pd.read_csv("group_results/metrics_GCE_groups_law.csv", index_col=0)
print(
    df.rename(columns=column_mapping)[column_mapping.values()].to_markdown(
        index=False, floatfmt=".2f"
    )
)

|     K |   Prob. Plaus. |   L1 |   L2 |   IsoForest |   LOF |   Log Dens. |
|------:|---------------:|-----:|-----:|------------:|------:|------------:|
|  2.00 |           0.92 | 0.53 | 0.33 |        0.02 |  1.09 |        1.68 |
|  3.00 |           0.91 | 0.59 | 0.36 |        0.01 |  1.08 |        1.63 |
|  4.00 |           0.94 | 0.45 | 0.28 |        0.04 |  1.06 |        2.02 |
|  5.00 |           0.93 | 0.44 | 0.27 |        0.03 |  1.06 |        1.99 |
|  6.00 |           0.93 | 0.45 | 0.28 |        0.04 |  1.06 |        2.07 |
|  7.00 |           0.94 | 0.48 | 0.30 |        0.03 |  1.06 |        1.97 |
|  8.00 |           0.96 | 0.47 | 0.29 |        0.04 |  1.06 |        2.05 |
|  9.00 |           0.95 | 0.47 | 0.29 |        0.03 |  1.06 |        2.02 |
| 10.00 |           0.96 | 0.47 | 0.29 |        0.03 |  1.06 |        2.03 |


In [11]:
df = pd.read_csv("group_results/metrics_GCE_k_means_groups_law.csv", index_col=0)
print(
    df.rename(columns=column_mapping)[column_mapping.values()].to_markdown(
        index=False, floatfmt=".2f"
    )
)

|     K |   Prob. Plaus. |   L1 |   L2 |   IsoForest |   LOF |   Log Dens. |
|------:|---------------:|-----:|-----:|------------:|------:|------------:|
|  2.00 |           0.95 | 0.61 | 0.38 |        0.03 |  1.07 |        1.94 |
|  3.00 |           0.98 | 0.57 | 0.39 |        0.04 |  1.04 |        2.25 |
|  4.00 |           0.99 | 0.50 | 0.33 |        0.05 |  1.03 |        2.42 |
|  5.00 |           0.99 | 0.57 | 0.38 |        0.05 |  1.03 |        2.39 |
|  6.00 |           0.99 | 0.48 | 0.31 |        0.06 |  1.02 |        2.56 |
|  7.00 |           1.00 | 0.48 | 0.31 |        0.06 |  1.02 |        2.62 |
|  8.00 |           1.00 | 0.49 | 0.31 |        0.06 |  1.02 |        2.54 |
|  9.00 |           1.00 | 0.49 | 0.30 |        0.06 |  1.02 |        2.59 |
| 10.00 |           1.00 | 0.48 | 0.30 |        0.06 |  1.02 |        2.55 |
